In [24]:
import pandas as pd
import numpy as np

df = pd.read_csv('npf_train.csv')


In [25]:
# Luodaan sarake class2 jossa on nonevent kun class4 == nonevent ja event kun class4 != nonevent

class2 = np.array(['event']*df.shape[0])
class2[df['class4']=='nonevent'] = 'nonevent'

df['class2'] = class2
df['class2'] = df['class2'].astype('category')

df.head()

,id,date,class4,partlybad,CO2168.mean,CO2168.std,CO2336.mean,CO2336.std,CO242.mean,CO242.std,...,T672.std,T84.mean,T84.std,UV_A.mean,UV_A.std,UV_B.mean,UV_B.std,CS.mean,CS.std,class2
0,1,2000-01-01,nonevent,False,384.462000,2.284996,384.164462,2.135062,385.274688,2.211695,...,0.525698,-12.422972,0.376324,1.635563,0.856948,0.026438,0.014617,0.003374,0.000733,nonev
1,2,2000-01-20,nonevent,False,374.884615,0.415185,374.703333,0.385179,375.621266,0.665720,...,0.373927,-8.351043,0.575679,1.441109,0.741088,0.022649,0.012479,0.001501,0.000572,nonev
2,3,2000-01-23,nonevent,False,373.496585,0.189497,373.382593,0.172958,373.961481,0.235107,...,0.965988,-9.651155,1.238891,2.677545,1.261612,0.044759,0.023748,0.000764,0.000048,nonev
3,4,2000-02-17,nonevent,False,378.600367,1.934180,378.464862,1.946536,379.785872,2.865022,...,0.122651,-0.829524,0.134191,2.261805,1.345651,0.030893,0.021903,0.002038,0.000751,nonev
4,5,2000-03-25,Ib,False,373.128684,1.096617,372.980000,1.047750,373.701830,1.259198,...,1.695622,-1.095864,2.090111,12.906779,7.022300,0.333523,0.239981,0.000662,0.000210,event


# Baseline malli
* Logistinen regressio
* https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
* ks. https://stackoverflow.com/questions/52670012/convergencewarning-liblinear-failed-to-converge-increase-the-number-of-iterati
* linkin takana vinkkejä 'failed to converge' varoitukseen
* https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-scaler

In [26]:
df.shape

(458, 105)

## Feature selection
- Select k-best (10 ~80% score)
https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection

Variance threshold vois olla myös hyvä

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

X = df.drop(['id', 'date', 'class4', 'partlybad', 'class2'], axis=1).to_numpy()
y = df['class2'].to_numpy()

scaler = StandardScaler().fit(X)

X_scaled = scaler.transform(X) # mean=0, std_deviation=1, Standard Normal Distribution

X_new = SelectKBest(f_classif, k=10).fit_transform(X_scaled, y) # feature selection, 10 parasta
#X_new.shape
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.33, random_state=42)

clf = LogisticRegression(random_state=0).fit(X_train, y_train)
score = clf.score(X_test, y_test)
print(f"score: {score}")

clf.predict(X_test) #output järkevä

score: 0.8026315789473685


array(['event', 'nonev', 'event', 'event', 'event', 'nonev', 'nonev',
       'event', 'event', 'event', 'nonev', 'event', 'nonev', 'event',
       'nonev', 'event', 'nonev', 'nonev', 'nonev', 'nonev', 'event',
       'event', 'event', 'event', 'event', 'nonev', 'event', 'nonev',
       'event', 'event', 'nonev', 'event', 'event', 'event', 'event',
       'nonev', 'event', 'nonev', 'nonev', 'nonev', 'nonev', 'nonev',
       'event', 'nonev', 'nonev', 'event', 'event', 'nonev', 'event',
       'event', 'event', 'nonev', 'event', 'event', 'nonev', 'nonev',
       'event', 'event', 'event', 'nonev', 'event', 'nonev', 'nonev',
       'nonev', 'nonev', 'event', 'nonev', 'nonev', 'event', 'event',
       'event', 'nonev', 'nonev', 'nonev', 'nonev', 'event', 'event',
       'nonev', 'event', 'event', 'event', 'nonev', 'nonev', 'nonev',
       'event', 'event', 'event', 'nonev', 'nonev', 'event', 'nonev',
       'event', 'nonev', 'event', 'event', 'nonev', 'nonev', 'event',
       'event', 'eve

In [28]:
clf.predict_proba(X_test)

array([[0.7536567 , 0.2463433 ],
       [0.12303236, 0.87696764],
       [0.77203834, 0.22796166],
       [0.87889858, 0.12110142],
       [0.92021802, 0.07978198],
       [0.05223386, 0.94776614],
       [0.42223109, 0.57776891],
       [0.60476531, 0.39523469],
       [0.77564027, 0.22435973],
       [0.62946833, 0.37053167],
       [0.3094074 , 0.6905926 ],
       [0.75606505, 0.24393495],
       [0.09473591, 0.90526409],
       [0.66453784, 0.33546216],
       [0.17516855, 0.82483145],
       [0.87513974, 0.12486026],
       [0.40371709, 0.59628291],
       [0.11622273, 0.88377727],
       [0.14547321, 0.85452679],
       [0.28941692, 0.71058308],
       [0.52121964, 0.47878036],
       [0.88688706, 0.11311294],
       [0.9069708 , 0.0930292 ],
       [0.58355113, 0.41644887],
       [0.8686498 , 0.1313502 ],
       [0.05232188, 0.94767812],
       [0.74954737, 0.25045263],
       [0.19787692, 0.80212308],
       [0.840073  , 0.159927  ],
       [0.63248286, 0.36751714],
       [0.

## Multilabel model
* Käyttää algoritmia 'newton-cg' -> tällä ei tullut convergence varoitusta

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df['class4'] = df['class4'].astype('category')
X2 = df.drop(['id', 'date', 'class4', 'partlybad', 'class2'], axis=1).to_numpy()
y2 = df['class4'].to_numpy()

scaler2 = StandardScaler().fit(X2)

X2_scaled = scaler2.transform(X2) # mean=0, std_deviation=1, Standard Normal Distribution

X_train, X_test, y_train, y_test = train_test_split(X2_scaled, y2, test_size=0.33, random_state=42)

clf2 = LogisticRegression(random_state=0, solver='newton-cg', multi_class='multinomial').fit(X_train, y_train)
score = clf2.score(X_test, y_test)
print(f"score: {score}")

labels = clf2.predict(X_test) #output järkevä

score: 0.618421052631579


In [30]:
probs = clf2.predict_proba(X_test)

In [34]:
for l in labels:
    for p in probs:
        maxp = max(p)
        
    print(f"{l}, prediction probability {maxp}")

II, prediction probability 0.9119203512949734
nonevent, prediction probability 0.9119203512949734
II, prediction probability 0.9119203512949734
Ib, prediction probability 0.9119203512949734
II, prediction probability 0.9119203512949734
Ia, prediction probability 0.9119203512949734
nonevent, prediction probability 0.9119203512949734
Ia, prediction probability 0.9119203512949734
Ib, prediction probability 0.9119203512949734
Ib, prediction probability 0.9119203512949734
nonevent, prediction probability 0.9119203512949734
Ib, prediction probability 0.9119203512949734
nonevent, prediction probability 0.9119203512949734
nonevent, prediction probability 0.9119203512949734
nonevent, prediction probability 0.9119203512949734
II, prediction probability 0.9119203512949734
nonevent, prediction probability 0.9119203512949734
nonevent, prediction probability 0.9119203512949734
nonevent, prediction probability 0.9119203512949734
nonevent, prediction probability 0.9119203512949734
nonevent, prediction